In [1]:
import numpy as np
import thingi10k
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, IntSlider, fixed
from voxel_dataset_generator.voxelization import Voxelizer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
thingi10k.init(variant='raw')
dataset = thingi10k.dataset()

object_info = next(iter(dataset))
file_name = object_info['file_path']

In [22]:
def plot_voxels_3d_cubes(voxels, voxel_pitch=1.0, colorscale='Viridis', max_voxels=10000):
    """Create a 3D mesh visualization of voxels as cubes.
    
    Args:
        voxels: Binary 3D array of voxel occupancy
        voxel_pitch: Size of each voxel cube in world space
        colorscale: Plotly colorscale name
        max_voxels: Maximum number of voxels to render (for performance)
    """
    occupied = np.where(voxels)
    x, y, z = occupied
    
    if len(x) == 0:
        return None
    
    vertices = []
    faces = []
    colors = []
    
    # Define cube vertices (unit cube scaled by pitch)
    cube_verts = np.array([
        [0,0,0], [1,0,0], [1,1,0], [0,1,0],
        [0,0,1], [1,0,1], [1,1,1], [0,1,1]
    ]) * voxel_pitch
    
    # Define cube faces as triangles
    cube_faces = np.array([
        [0,1,2], [0,2,3],  # Bottom
        [4,5,6], [4,6,7],  # Top
        [0,1,5], [0,5,4],  # Front
        [2,3,7], [2,7,6],  # Back
        [0,3,7], [0,7,4],  # Left
        [1,2,6], [1,6,5]   # Right
    ])
    
    # Create mesh for each voxel
    # Position each cube at (xi*pitch, yi*pitch, zi*pitch)
    for xi, yi, zi in zip(x, y, z):
        verts = cube_verts + np.array([xi, yi, zi]) * voxel_pitch
        base_idx = len(vertices)
        vertices.extend(verts)
        faces.extend(cube_faces + base_idx)
        colors.extend([zi] * 8)  # Color by z-coordinate
    
    vertices = np.array(vertices)
    faces = np.array(faces)
    colors = np.array(colors)
    
    return go.Mesh3d(
        x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
        i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
        intensity=colors, colorscale=colorscale,
        showscale=False, opacity=0.9
    )

def plot_voxels_matplotlib(ax, voxels, voxel_pitch=1.0, cmap='viridis', alpha=0.7):
    """Create a 3D voxel plot using matplotlib with color by z-height.
    
    Args:
        ax: Matplotlib 3D axis
        voxels: Binary 3D array of voxel occupancy
        voxel_pitch: Size of each voxel cube in world space
        cmap: Colormap name for coloring by z-coordinate
        alpha: Transparency
    """
    import matplotlib.cm as cm
    import matplotlib.colors as mcolors
    
    # Get the colormap
    colormap = cm.get_cmap(cmap)
    
    # Create a color array for each voxel based on z-coordinate
    shape = voxels.shape
    colors = np.empty(voxels.shape, dtype=object)
    
    # Normalize z-coordinates to [0, 1] for colormap
    z_max = shape[2]
    
    for x in range(shape[0]):
        for y in range(shape[1]):
            for z in range(shape[2]):
                if voxels[x, y, z]:
                    # Color based on z-coordinate (height)
                    normalized_z = z / z_max
                    rgba = colormap(normalized_z)
                    # Apply alpha
                    rgba = (*rgba[:3], alpha)
                    colors[x, y, z] = rgba
    
    # Plot voxels with color array
    ax.voxels(voxels, facecolors=colors, edgecolor='k', linewidth=0.1)
    
    # Scale the axes
    ax.set_xlim(0, shape[0])
    ax.set_ylim(0, shape[1])
    ax.set_zlim(0, shape[2])
    
    # Remove axis labels and ticks for cleaner look
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_zlabel('')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    
    # Set aspect ratio to be equal
    ax.set_box_aspect([1, 1, 1])

def visualize_comparison(res_level, min_resolution=32, show_interactive=True,
                         print_console=True, save_image=False, output_path=None):
    """Visualize upsampled vs direct voxelization at a given resolution level.

    Args:
        res_level: Resolution level (0-5)
        min_resolution: Minimum resolution (default: 32)
        show_interactive: If True, display interactive 3D plots (default: True)
        print_console: If True, print results to console (default: True)
        save_image: If True, save a static image (default: False)
        output_path: Path to save image (default: f'comparison_level_{res_level}.png')
    """
    min_res_level = int(np.log2(min_resolution))

    # Create voxelizers
    base_resolution = 2**(min_res_level + res_level)
    high_resolution = 2**(min_res_level + res_level + 1)

    voxelizer_base = Voxelizer(base_resolution, solid=True)
    voxelizer_high = Voxelizer(high_resolution, solid=True)

    # Voxelize at both resolutions and get metadata
    voxel_grid_base, metadata_base = voxelizer_base.voxelize_file(file_name)
    voxel_grid_high, metadata_high = voxelizer_high.voxelize_file(file_name)

    # Get voxel pitches from metadata
    pitch_base = metadata_base['voxel_pitch']
    pitch_high = metadata_high['voxel_pitch']

    # Upsample the base resolution by 2x in each dimension
    voxel_grid_upsampled = voxel_grid_base.repeat(2, 0).repeat(2, 1).repeat(2, 2)
    # Upsampled pitch is half of base pitch (since we doubled resolution)
    pitch_upsampled = pitch_base / 2.0

    # Calculate difference
    diff_percentage = (voxel_grid_upsampled ^ voxel_grid_high).sum() / voxel_grid_high.sum() * 100

    # Print to console
    if print_console:
        print(f"\nResolution Level: {res_level}")
        print(f"Base resolution: {base_resolution}³, pitch: {pitch_base:.6f}")
        print(f"High resolution: {high_resolution}³, pitch: {pitch_high:.6f}")
        print(f"Upsampled pitch: {pitch_upsampled:.6f}")
        print(f"Pitch ratio (upsampled/direct): {pitch_upsampled/pitch_high:.6f} (should be ~1.0)")
        print(f"\nDifference: {diff_percentage:.4f}%")
        print(f"Number of voxels in upsampled: {voxel_grid_upsampled.sum()}")
        print(f"Number of voxels in direct: {voxel_grid_high.sum()}")
        print(f"\nOriginal bbox extents: {metadata_high['original_bbox_extents']}")

    # Create interactive visualization with Plotly
    if show_interactive:
        # Create subplots with synchronized cameras
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=(
                f'Upsampled ({base_resolution}→{high_resolution})<br>pitch={pitch_upsampled:.6f}',
                f'Direct Voxelization ({high_resolution})<br>pitch={pitch_high:.6f}'
            ),
            specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
            horizontal_spacing=0.0,
            vertical_spacing=0.
        )

        # Recompute hollow visualisation for display
        voxelizer_base = Voxelizer(base_resolution, solid=False)
        voxelizer_high = Voxelizer(high_resolution, solid=False)

        # Voxelize at both resolutions and get metadata
        voxel_grid_base_hollow, _ = voxelizer_base.voxelize_file(file_name)
        voxel_grid_high_hollow, _ = voxelizer_high.voxelize_file(file_name)

        # Upsample the hollow version for visualization
        voxel_grid_upsampled_hollow = voxel_grid_base_hollow.repeat(2, 0).repeat(2, 1).repeat(2, 2)

        # Create mesh traces for both visualizations with correct pitches
        trace_upsampled = plot_voxels_3d_cubes(voxel_grid_upsampled_hollow, pitch_upsampled, 'Viridis', max_voxels=10000)
        trace_direct = plot_voxels_3d_cubes(voxel_grid_high_hollow, pitch_high, 'Plasma', max_voxels=10000)

        # Add traces to subplots
        if trace_upsampled:
            fig.add_trace(trace_upsampled, row=1, col=1)
        if trace_direct:
            fig.add_trace(trace_direct, row=1, col=2)

        # Calculate world-space bounds
        max_extent = max(metadata_high['original_bbox_extents'])

        fig.update_layout(
            title_text=f'Resolution Level {res_level} | Difference: {diff_percentage:.2f}%<br>' +
                       f'Base Resolution: {base_resolution}³, High Resolution: {high_resolution}³',
            showlegend=False,
            height=None,
            width=2000,
            autosize=True,
            margin=dict(t=140, b=20, l=20, r=20),  # Increased top margin to prevent title overlap
            scene=dict(
                xaxis=dict(range=[0, max_extent], title='X', visible=True),
                yaxis=dict(range=[0, max_extent], title='Y', visible=True),
                zaxis=dict(range=[0, max_extent], title='Z', visible=True),
                # aspectmode='cube',
                camera=dict(eye=dict(x=1., y=-1., z=0.5),
                projection=dict(type='orthographic')  # Rotated 180° around up axis
                )
            ),
            scene2=dict(
                xaxis=dict(range=[0, max_extent], title='X', visible=True),
                yaxis=dict(range=[0, max_extent], title='Y', visible=True),
                zaxis=dict(range=[0, max_extent], title='Z', visible=True),
                # aspectmode='cube',
                camera=dict(eye=dict(x=1., y=-1., z=0.5),
                            projection=dict(type='orthographic')  # Rotated 180° around up axis
                )
            )
        )

        fig.show()
        
        # Show sampling info
        if print_console:
            upsampled_count = voxel_grid_upsampled.sum()
            direct_count = voxel_grid_high.sum()
            if upsampled_count > 10000:
                print(f"Note: Upsampled view shows 10,000 of {upsampled_count} voxels (sampled for performance)")
            if direct_count > 10000:
                print(f"Note: Direct view shows 10,000 of {direct_count} voxels (sampled for performance)")

    # Save static image with matplotlib
    if save_image:
        # Recompute hollow visualisation for display
        voxelizer_base_hollow = Voxelizer(base_resolution, solid=False)
        voxelizer_high_hollow = Voxelizer(high_resolution, solid=False)

        # Voxelize at both resolutions
        voxel_grid_base_hollow, _ = voxelizer_base_hollow.voxelize_file(file_name)
        voxel_grid_high_hollow, _ = voxelizer_high_hollow.voxelize_file(file_name)

        # Upsample the hollow version for visualization
        voxel_grid_upsampled_hollow = voxel_grid_base_hollow.repeat(2, 0).repeat(2, 1).repeat(2, 2)

        # Create matplotlib figure
        fig_mpl, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7), subplot_kw={'projection': '3d'})
        
        fig_mpl.suptitle(
            f'Resolution Level {res_level} | Difference: {diff_percentage:.2f}%\n'
            f'Base Resolution: {base_resolution}³, High Resolution: {high_resolution}³',
            fontsize=14, fontweight='bold'
        )
        
        # Plot upsampled (using 'plasma' colormap for more visual differentiation)
        ax1.set_title(f'Upsampled ({base_resolution}→{high_resolution})\npitch={pitch_upsampled:.6f}')
        plot_voxels_matplotlib(ax1, voxel_grid_upsampled_hollow, pitch_upsampled, cmap='plasma', alpha=0.7)
        
        # Plot direct (using 'viridis' colormap for visual differentiation from upsampled)
        ax2.set_title(f'Direct Voxelization ({high_resolution})\npitch={pitch_high:.6f}')
        plot_voxels_matplotlib(ax2, voxel_grid_high_hollow, pitch_high, cmap='viridis', alpha=0.7)
        
        # Set viewing angle - rotate 180 degrees about up axis (azim += 180)
        for ax in [ax1, ax2]:
            ax.view_init(elev=20, azim=225)  # 45 + 180 = 225
        
        plt.tight_layout()
        
        if output_path is None:
            output_path = f'comparison_level_{res_level}.png'
        
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        if print_console:
            print(f"Static image saved to: {output_path}")

In [23]:
# Interactive visualization with slider
min_resolution = 32
max_res_level = 5

# Wrapper function that excludes output_path from the widget
def interactive_visualize(res_level, min_resolution=32):
    visualize_comparison(
        res_level=res_level,
        min_resolution=min_resolution,
        show_interactive=True,
        print_console=True,
        save_image=False,
        output_path=None
    )

_ = interact(
    interactive_visualize,
    res_level=IntSlider(
        min=0, 
        max=max_res_level, 
        step=1, 
        value=0, 
        description='Res Level:',
        continuous_update=False
    ),
    min_resolution=fixed(min_resolution)
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Res Level:', max=5), Output()),…

In [ ]:
# Generate static HTML files for all resolution levels (fast!)
import os

# Create output directory if it doesn't exist
output_dir = 'comparison_images'
os.makedirs(output_dir, exist_ok=True)

min_resolution = 32
max_res_level = 5

print(f"Generating HTML files for resolution levels 0 to {max_res_level}...")
print(f"Output directory: {output_dir}/")
print("=" * 80)

for res_level in range(max_res_level + 1):
    output_path = os.path.join(output_dir, f'comparison_level_{res_level}.html')
    
    # We'll need to manually create the plot and save it as HTML
    min_res_level = int(np.log2(min_resolution))
    base_resolution = 2**(min_res_level + res_level)
    high_resolution = 2**(min_res_level + res_level + 1)
    
    voxelizer_base = Voxelizer(base_resolution, solid=True)
    voxelizer_high = Voxelizer(high_resolution, solid=True)
    
    # Voxelize at both resolutions and get metadata
    voxel_grid_base, metadata_base = voxelizer_base.voxelize_file(file_name)
    voxel_grid_high, metadata_high = voxelizer_high.voxelize_file(file_name)
    
    # Get voxel pitches from metadata
    pitch_base = metadata_base['voxel_pitch']
    pitch_high = metadata_high['voxel_pitch']
    
    # Upsample the base resolution by 2x in each dimension
    voxel_grid_upsampled = voxel_grid_base.repeat(2, 0).repeat(2, 1).repeat(2, 2)
    pitch_upsampled = pitch_base / 2.0
    
    # Calculate difference
    diff_percentage = (voxel_grid_upsampled ^ voxel_grid_high).sum() / voxel_grid_high.sum() * 100
    
    print(f"\nResolution Level: {res_level}")
    print(f"Base resolution: {base_resolution}³ -> High resolution: {high_resolution}³")
    print(f"Difference: {diff_percentage:.4f}%")
    
    # Create subplots with synchronized cameras
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            f'Upsampled ({base_resolution}→{high_resolution})<br>pitch={pitch_upsampled:.6f}',
            f'Direct Voxelization ({high_resolution})<br>pitch={pitch_high:.6f}'
        ),
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        horizontal_spacing=0.05,
        vertical_spacing=0.15
    )
    
    # Recompute hollow visualisation for display
    voxelizer_base_hollow = Voxelizer(base_resolution, solid=False)
    voxelizer_high_hollow = Voxelizer(high_resolution, solid=False)
    
    # Voxelize at both resolutions and get metadata
    voxel_grid_base_hollow, _ = voxelizer_base_hollow.voxelize_file(file_name)
    voxel_grid_high_hollow, _ = voxelizer_high_hollow.voxelize_file(file_name)
    
    # Upsample the hollow version for visualization
    voxel_grid_upsampled_hollow = voxel_grid_base_hollow.repeat(2, 0).repeat(2, 1).repeat(2, 2)
    
    # Create mesh traces for both visualizations with correct pitches
    trace_upsampled = plot_voxels_3d_cubes(voxel_grid_upsampled_hollow, pitch_upsampled, 'Viridis', max_voxels=10000)
    trace_direct = plot_voxels_3d_cubes(voxel_grid_high_hollow, pitch_high, 'Plasma', max_voxels=10000)
    
    # Add traces to subplots
    if trace_upsampled:
        fig.add_trace(trace_upsampled, row=1, col=1)
    if trace_direct:
        fig.add_trace(trace_direct, row=1, col=2)
    
    # Calculate world-space bounds
    max_extent = max(metadata_high['original_bbox_extents'])
    
    fig.update_layout(
        title_text=f'Resolution Level {res_level} | Difference: {diff_percentage:.2f}%<br>' +
                   f'Base Resolution: {base_resolution}³, High Resolution: {high_resolution}³',
        showlegend=False,
        height=700,
        width=None,  # Full width
        margin=dict(t=120, b=20, l=20, r=20),  # Increased top margin to prevent title overlap
        scene=dict(
            xaxis=dict(range=[0, max_extent], title='X', visible=False),
            yaxis=dict(range=[0, max_extent], title='Y', visible=False),
            zaxis=dict(range=[0, max_extent], title='Z', visible=False),
            aspectmode='cube',
            camera=dict(eye=dict(x=-1.5, y=-1.5, z=1.5))  # Rotated 180° around up axis
        ),
        scene2=dict(
            xaxis=dict(range=[0, max_extent], title='X', visible=False),
            yaxis=dict(range=[0, max_extent], title='Y', visible=False),
            zaxis=dict(range=[0, max_extent], title='Z', visible=False),
            aspectmode='cube',
            camera=dict(eye=dict(x=-1.5, y=-1.5, z=1.5))  # Rotated 180° around up axis
        )
    )
    
    # Save as HTML (very fast!)
    fig.write_html(output_path)
    print(f"HTML file saved to: {output_path}")
    print("=" * 80)

print(f"\nAll HTML files saved to {output_dir}/")
print("You can open them in your browser for interactive viewing!")